<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/p011_Transfer_Learning_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.2


# Objectif
 * Utilisez `tf.keras.applications.Xception()` pour loader un modèle pretrained
 * Renseigner l'argument `weights = 'ImageNet'` afin de choisir sur le dataset d'entraînement du pretrained model
 * On peut également renseigner le format de l'input d'image avec `shape = (150,3,3)`
 * Evidemment, on ne veut pas du dernier layer, car on veut adapter le modèle à nos données, on fait donc `include_top=False`
 * Freezer les poids du modèle
 * Rajouter un GlobalPoolingAverage2d
 * Rajouter une couche dense au top du modèle
 * Entraîner le modèle sur de nouvelles données via une custom loop, on prétend qu'on a à dispotion un dataset tensorflow

In [2]:
# Récupérattion du pretrained model xception entrainé sur imagenet
base_model = tf.keras.applications.Xception(include_top=False,
                                            weights="imagenet",
                                            input_shape=(150,150,3))

base_model.trainable = False

83697664/83683744 [==============================] - 1s 0us/step


In [3]:
# Création du nouveau modèle
input = tf.keras.layers.Input(shape=(150,150,3),dtype="float32")
x = base_model(input)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(input,output)

In [4]:
# Instanciation de l'une loss et optmizer
loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()
epoch = 5

In [ ]:
# La Custom Loop

for _ in range(epoch):

  for batch_data,batch_target in dataset:

    # Mise à jour des poids
    with tf.GradientTape() as tape:

      # Predictions
      predictions = model(batch_data)
      loss_value = loss(batch_target,predictions)

    # Calcul des gradients
    # Mise à jour des poids
    grads = tape.gradient(loss_value,model.trainable_weights)
    optimizer.apply_gradients(zip(grads,model.trainable_weights))


In [ ]:
# Quand on boucle sur le dataset on peut mettre un tuple inputs,labels